In [95]:
import pandas as pd
from tqdm import tqdm
import os
test_names, test_texts = [], []
test_texts_list = []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/test'))):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('../input/feedback-prize-2021/test/' + f, 'r', encoding='utf-8').read())
for t in test_texts:
    test_texts_list.append(t.split())
test_text_df = pd.DataFrame({'id': test_names, 'text_list': test_texts_list})

100%|██████████| 5/5 [00:00<00:00, 3034.51it/s]


In [96]:
from datasets import Dataset

test_datasets = Dataset.from_pandas(test_text_df)

In [97]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('../input/longformer-preprocess/best_model/longformer-preprocess_fold'+str(0)+'/', add_prefix_space=True)
eval_datasets = {'word_ids':[], 'id':[]}
def preparing_test_dataset(examples):
    encoding = tokenizer(examples['text_list'], truncation=True, padding=False, max_length = 2048, is_split_into_words=True)
    total= len(encoding['input_ids'])
    for i in range(total):
        word_idx = encoding.word_ids(batch_index=i)
        eval_datasets['word_ids'].append(word_idx)
        eval_datasets['id'].append(examples['id'][i])
    return encoding

tokenized_test_datasets = test_datasets.map(preparing_test_dataset, batched=True, remove_columns=test_datasets.column_names)

Didn't find file ../input/longformer-baseline-kfold/best_model/longformer-baseline_fold0/added_tokens.json. We won't load it.
loading file ../input/longformer-baseline-kfold/best_model/longformer-baseline_fold0/vocab.json
loading file ../input/longformer-baseline-kfold/best_model/longformer-baseline_fold0/merges.txt
loading file ../input/longformer-baseline-kfold/best_model/longformer-baseline_fold0/tokenizer.json
loading file None
loading file ../input/longformer-baseline-kfold/best_model/longformer-baseline_fold0/special_tokens_map.json
loading file ../input/longformer-baseline-kfold/best_model/longformer-baseline_fold0/tokenizer_config.json


  0%|          | 0/1 [00:00<?, ?ba/s]

In [98]:
tokenized_test_datasets

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 5
})

In [99]:
from transformers import AutoModelForTokenClassification, AutoConfig, Trainer, DataCollatorForTokenClassification, TrainingArguments
import numpy as np
import torch
data_collator = DataCollatorForTokenClassification(tokenizer)
all_predictions = 0
training_args = TrainingArguments(per_device_eval_batch_size=1, output_dir = '../input')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
for fold in range(5):
    model_path = '../input/longformer-preprocess/best_model/longformer-preprocess_fold'+str(fold)+'/'
    config = AutoConfig.from_pretrained(model_path)
    model = AutoModelForTokenClassification.from_pretrained(model_path, config = config)
    
    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset=None,
        eval_dataset=tokenized_test_datasets,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    predictions, _, _ = trainer.predict(test_dataset = tokenized_test_datasets)
    print("shape of prediciton", predictions.shape)
    predictions = predictions.astype(np.float32)
    predictions = predictions/5
    all_predictions+=predictions
    torch.cuda.empty_cache()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file ../input/longformer-baseline-kfold/best_model/longformer-baseline_fold0/config.json
Model config LongformerConfig {
  "_name_or_path": "../input/longformer-baseline-kfold/best_model/longformer-baseline_fold0/",
  "architectures": [
    "LongformerForTokenClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2lab

Input ids are automatically padded from 792 to 1024 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 749 to 1024 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 466 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 1292 to 1536 to be a multiple of `config.attention_window`: 512
loading configuration file ../input/longformer-baseline-kfold/best_model/longformer-baseline_fold1/config.json
Model config LongformerConfig {
  "_name_or_path": "../input/longformer-baseline-kfold/best_model/longformer-baseline_fold1/",
  "architectures": [
    "LongformerForTokenClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "g

shape of prediciton (5, 1292, 15)


All model checkpoint weights were used when initializing LongformerForTokenClassification.

All the weights of LongformerForTokenClassification were initialized from the model checkpoint at ../input/longformer-baseline-kfold/best_model/longformer-baseline_fold1/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LongformerForTokenClassification for predictions without further training.
***** Running Prediction *****
  Num examples = 5
  Batch size = 1
Input ids are automatically padded from 431 to 512 to be a multiple of `config.attention_window`: 512


Input ids are automatically padded from 792 to 1024 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 749 to 1024 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 466 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 1292 to 1536 to be a multiple of `config.attention_window`: 512
loading configuration file ../input/longformer-baseline-kfold/best_model/longformer-baseline_fold2/config.json
Model config LongformerConfig {
  "_name_or_path": "../input/longformer-baseline-kfold/best_model/longformer-baseline_fold2/",
  "architectures": [
    "LongformerForTokenClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "g

...

Input ids are automatically padded from 792 to 1024 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 749 to 1024 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 466 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 1292 to 1536 to be a multiple of `config.attention_window`: 512
loading configuration file ../input/longformer-baseline-kfold/best_model/longformer-baseline_fold4/config.json
Model config LongformerConfig {
  "_name_or_path": "../input/longformer-baseline-kfold/best_model/longformer-baseline_fold4/",
  "architectures": [
    "LongformerForTokenClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "g

shape of prediciton (5, 1292, 15)


All model checkpoint weights were used when initializing LongformerForTokenClassification.

All the weights of LongformerForTokenClassification were initialized from the model checkpoint at ../input/longformer-baseline-kfold/best_model/longformer-baseline_fold4/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LongformerForTokenClassification for predictions without further training.
***** Running Prediction *****
  Num examples = 5
  Batch size = 1
Input ids are automatically padded from 431 to 512 to be a multiple of `config.attention_window`: 512


Input ids are automatically padded from 792 to 1024 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 749 to 1024 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 466 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 1292 to 1536 to be a multiple of `config.attention_window`: 512


shape of prediciton (5, 1292, 15)


In [101]:
from collections import defaultdict

def label_dict():
    train = pd.read_csv('../input/feedback-prize-2021/train.csv')
    classes = train.discourse_type.unique().tolist()
    tags = defaultdict()
    for i, c in enumerate(classes):
        tags[f'B-{c}'] = i
        tags[f'I-{c}'] = i + len(classes)
    tags[f'O'] = len(classes) * 2
    tags[f'Special'] = -100
    l2i = dict(tags)
    i2l = defaultdict()
    for k, v in l2i.items(): 
        i2l[v] = k
    i2l[-100] = 'Special'
    i2l = dict(i2l)
    N_LABELS = len(i2l) - 1 # not accounting for -100
    return i2l, l2i, N_LABELS

In [102]:
def link_evidence(oof):
  if not len(oof):
    return oof
  
  def jn(pst, start, end):
    return " ".join([str(x) for x in pst[start:end]])
  
  thresh = 1
  idu = oof['id'].unique()
  eoof = oof[oof['class'] == "Evidence"]
  neoof = oof[oof['class'] != "Evidence"]
  eoof.index = eoof[['id', 'class']]
  for thresh2 in range(26, 27, 1):
    retval = []
    for idv in tqdm(idu, desc='link_evidence', leave=False):
      for c in ['Evidence']:
        q = eoof[(eoof['id'] == idv)]
        if len(q) == 0:
          continue
        pst = []
        for r in q.itertuples():
          pst = [*pst, -1,  *[int(x) for x in r.predictionstring.split()]]
        start = 1
        end = 1
        for i in range(2, len(pst)):
          cur = pst[i]
          end = i
          if  ((cur == -1) and ((pst[i + 1] > pst[end - 1] + thresh) or (pst[i + 1] - pst[start] > thresh2))):
            retval.append((idv, c, jn(pst, start, end)))
            start = i + 1
        v = (idv, c, jn(pst, start, end + 1))
        retval.append(v)
    roof = pd.DataFrame(retval, columns=['id', 'class', 'predictionstring'])
    roof = roof.merge(neoof, how='outer')
    return roof

In [106]:
def find_max_label(word_prediction_score):
    x = np.sum(word_prediction_score, axis=0)
    max_label = np.argmax(x, axis=-1)
    return max_label

In [118]:
def postprocess_fb_predictions2(
    eval_datasets,
    predictions,
):
    proba_thresh = {
        "Lead": 0.687,
        "Position": 0.537,
        "Evidence": 0.637,
        "Claim": 0.537,
        "Concluding Statement": 0.687,
        "Counterclaim": 0.537,
        "Rebuttal": 0.537,
    }
    #discourse length threshold
    min_thresh = {
        "Lead": 9,
        "Position": 5,
        "Evidence": 14,
        "Claim": 3,
        "Concluding Statement": 11,
        "Counterclaim": 6,
        "Rebuttal": 4,
    }
    print(predictions.shape)
    softmax = torch.nn.Softmax(dim=-1)
    predictions = torch.tensor(predictions)
    pred_score = softmax(predictions)
    pred_score = pred_score.numpy()
    i2l, _, _ = label_dict()
    all_prediction = []
    all_pred_score=[]
    for k, label_pred_score in tqdm(enumerate(pred_score), desc = "post-processing"):
      each_prediction = []
      each_prediction_score = []
      word_ids = eval_datasets['word_ids'][k]
      previous_word_idx = -1
      word_prediction_score=[]
      for idx, word_idx in enumerate(word_ids):
        if word_idx == None:
          continue
        
        elif word_idx != previous_word_idx:
          if len(word_prediction_score)!=0:
            # find label which have the most score label following each tokens including in one word
            max_label = find_max_label(word_prediction_score)
            word_prediction_score = [word_prediction_score[i][max_label] for i in range(len(word_prediction_score))]
            each_prediction_score.append(word_prediction_score)
            each_prediction.append(i2l[max_label])
          
          previous_word_idx = word_idx
          word_prediction_score=[]
          word_prediction_score.append(label_pred_score[idx])
        
        else:
          word_prediction_score.append(label_pred_score[idx])
      
      max_label = find_max_label(word_prediction_score)
      word_prediction_score = [word_prediction_score[i][max_label] for i in range(len(word_prediction_score))]  
      each_prediction_score.append(word_prediction_score)
      each_prediction.append(i2l[max_label])
      
      all_prediction.append(each_prediction)
      all_pred_score.append(each_prediction_score)
    final_pred = []
    for i in range(len(eval_datasets['id'])):
      idx = eval_datasets['id'][i]
      pred = all_prediction[i]
      pred_score = all_pred_score[i]
      j=0
      while j < len(pred):
        cls = pred[j]
        if cls =='O': 
            j+=1
        else: 
            cls = cls.replace('B', 'I')
        end = j+1
        while end < len(pred) and pred[end] == cls:
            end +=1
        final_pred_score = []
        for item in pred_score[j:end]:
          final_pred_score.extend(item)
        if cls != 'O' and cls!='' and sum(final_pred_score)/len(final_pred_score)>=proba_thresh[cls.replace('I-', '')] and len(final_pred_score)>=min_thresh[cls.replace('I-', '')]:
            final_pred.append((idx, cls.replace('I-', ''), ' '.join(map(str, list(range(j, end))))))
        j = end
    oof = pd.DataFrame(final_pred)
    oof.columns = ['id', 'class', 'predictionstring']
    
    oof = link_evidence(oof)
    return oof

In [119]:
oof = postprocess_fb_predictions2(eval_examples=None, eval_datasets=eval_datasets, predictions=all_predictions)

(5, 1292, 15)


5it [00:00, 77.43it/s]


In [111]:
oof.to_csv("submission.csv", index=False)